# Parsing Puzzle

In [21]:
import io


def parse_pips_puzzle(filename):
    with open(filename, 'r') as f:
        return parse_pips_puzzle_helper(f)
    
def parse_pips_puzzle_string(puzzle_string):
    return parse_pips_puzzle_helper(io.StringIO(puzzle_string))

def parse_pips_puzzle_helper(buffer):
    # read rows/cols
    n = int(buffer.readline().strip())

    # read board matrix
    board = []
    for _ in range(n):
        row = list(buffer.readline().strip())
        line = []
        for c in row:
            if c == '…':
                line.extend(['.', '.', '.'])
            else:
                line.append(c)
        
        board.append(line)

    # read number of constraints
    c = int(buffer.readline().strip())

    constraints = {}
    for _ in range(c):
        parts = buffer.readline().strip().split()
        letter = parts[0]
        ctype = parts[1]     # EQ, NEQ, LT, GT, SUM
        # Constraints with NO value
        if ctype in ("EQ", "NEQ"):
            value = None
        # Constraints with a numeric value
        elif ctype in ("SUM", "LT", "GT"):
            if len(parts) < 3:
                raise ValueError(f"Constraint '{parts}' missing numeric value")
            value = int(parts[2])
        else:
            raise ValueError("Unknown constraint type: " + ctype)

        constraints[letter] = (ctype, value)

    # read number of dominos
    m = int(buffer.readline().strip())

    dominos = []
    for _ in range(m):
        a, b = buffer.readline().strip().split()
        dominos.append((int(a), int(b)))

    return n, board, constraints, dominos

Below, can input file containing the puzzle and it will be parsed and stored

In [7]:
n, board, constraints, dominos = parse_pips_puzzle('pips1.txt')
print("Board size:", n)
print("Board:")
for row in board:
    print(' '.join(row))
print("Constraints:", constraints)
print("Dominos:", dominos)

Board size: 4
Board:
A # # B
. . . B
. . C C
. . C .
Constraints: {'A': ('SUM', 3), 'B': ('SUM', 11), 'C': ('SUM', 15)}
Dominos: [(5, 1), (6, 5), (3, 0), (5, 5)]


# Writing Variables and Constraints

## Variables

We model the Pips puzzle as a constraint satisfaction problem using two types of decision variables.
First, each domino is represented by a single integer variable place[d] whose domain consists of all legal placements of that domino on the board; each placement encodes a row, a column, and an orientation, and immediately determines which two cells the domino covers and which pip values it contributes.
Second, for each valid board cell we introduce an integer variable cell_value[r][c] representing the pip value assigned to that cell.
The link between these two variable types will be enforced by conditional constraints: whenever a domino chooses a particular placement, it sets the pip values of the two cells it covers accordingly.

In [8]:
from ortools.sat.python import cp_model

def define_variables(model, n, board, dominos):
    m = len(dominos)
    maxpip = max(max(d) for d in dominos)  # highest pip value

    # 1. Generate all valid placements
    placements = [[] for _ in range(m)]

    for d, (v0, v1) in enumerate(dominos):
        # Horizontal placements
        for r in range(n):
            for c in range(n - 1):
                if board[r][c] != '.' and board[r][c+1] != '.':
                    placements[d].append((r, c, 0, v0, v1))  # forward
                    placements[d].append((r, c, 1, v1, v0))  # reverse

        # Vertical placements
        for r in range(n - 1):
            for c in range(n):
                if board[r][c] != '.' and board[r+1][c] != '.':
                    placements[d].append((r, c, 2, v0, v1))  # forward
                    placements[d].append((r, c, 3, v1, v0))  # reverse

    # 2. Create domino placement variables
    place = []
    for d in range(m):
        K = len(placements[d])
        var = model.NewIntVar(0, K - 1, f"place_{d}")
        place.append(var)

    # 3. Create cell pip-value variables
    cell_value = [[None]*n for _ in range(n)]

    for r in range(n):
        for c in range(n):
            if board[r][c] == '.':
                continue   # unusable cell -> no variable
            cell_value[r][c] = model.NewIntVar(0, maxpip, f"cell_{r}_{c}")

    return place, placements, cell_value, maxpip


## Constraints

For our constraints, we need to ensure that:
1. Every valid cell has exactly one variable.
2. Every domino has been placed.
3. The solution is valid and dominos have been placed correctly (i.e. can't split up the halves of the domino)
4. The constraints given in the Pips problem have been met.

1. Ensuring that every valid cell has exactly one variable:
- For each domino placement we create a Boolean indicator variable.
- For each cell, we gather all domino placements that physically cover that cell.
- Using AddExactlyOne, we enforce that exactly one of those placements is chosen.
- This ensures every valid cell is covered once and only once by dominos.

In [9]:
def add_cell_coverage_constraints(model, n, board, place, placements, cell_value):
    m = len(placements)

    # Create Boolean indicator variables for each placement
    is_place = []
    for d in range(m):
        K = len(placements[d])
        is_place_d = []
        for k in range(K):
            b = model.NewBoolVar(f"is_place_{d}_{k}")
            model.Add(place[d] == k).OnlyEnforceIf(b)
            model.Add(place[d] != k).OnlyEnforceIf(b.Not())
            is_place_d.append(b)
        is_place.append(is_place_d)

    # For every valid cell, enforce exactly 1 covering domino
    for r in range(n):
        for c in range(n):
            if board[r][c] == '.':
                continue  # unusable cell, skip

            covering_literals = []

            # Find all domino placements that cover this cell
            for d in range(m):
                for k, (rr, cc, o, v0, v1) in enumerate(placements[d]):
                    # Domino placement covers two cells depending on orientation
                    if o == 0:  # horizontal forward
                        cells = [(rr, cc), (rr, cc+1)]
                    elif o == 1:  # horizontal reverse
                        cells = [(rr, cc), (rr, cc+1)]
                    elif o == 2:  # vertical forward
                        cells = [(rr, cc), (rr+1, cc)]
                    else:  # o == 3, vertical reverse
                        cells = [(rr, cc), (rr+1, cc)]

                    # If this placement covers (r,c), include it
                    if (r, c) in cells:
                        covering_literals.append(is_place[d][k])

            # Exactly one domino covers this cell
            model.AddExactlyOne(covering_literals)

    return is_place


2. Ensuring that every domino is placed exactly once.
In the CP-SAT model, each domino is represented by a single integer variable place[d] whose domain is the set of all legal placements for that domino. Because this variable must take exactly one value in any solution, we no longer need a separate “exactly once” constraint for each domino.

3. The solution is valid and dominos have been placed correctly (i.e. can't split up the halves of the domino).
We can encode this with implication. I.e. if a domino is placed, the exact two cells must contain its two halves.

We already have a constraint saying that each domino can only be placed once, and these two in coordination ensure that our solution is feasible.
Although coverage constraints ensure that each cell is occupied exactly once, they do not specify which pip value appears in each cell. To guarantee that each domino contributes the correct two pip values according to the chosen placement, we add a channeling constraint: whenever place[d] selects a placement k, the pip values of the two cells covered by that placement are fixed to the domino’s values. This prevents dominos from being “split,” rotated inconsistently, or assigned incorrect values.

In [10]:
def add_domino_value_constraints(model, n, placements, place, is_place, cell_value):
    m = len(placements)

    for d in range(m):                 # each domino
        for k, (r, c, o, v0, v1) in enumerate(placements[d]):

            # If placement k is chosen, enforce value on first cell
            model.Add(cell_value[r][c] == v0).OnlyEnforceIf(is_place[d][k])

            # Determine the second cell based on orientation
            if o == 0 or o == 1:       # horizontal (forward or reverse)
                rr, cc = r, c+1
            else:                      # o == 2 or 3 (vertical)
                rr, cc = r+1, c

            # If placement k is chosen, enforce value on second cell
            model.Add(cell_value[rr][cc] == v1).OnlyEnforceIf(is_place[d][k])



4. The constraints given in the Pips problem have been met.


In [11]:
def add_region_EQ(model, cells, cell_value):
    if len(cells) <= 1:
        return
    (r0, c0) = cells[0]
    base = cell_value[r0][c0]
    for (r, c) in cells[1:]:
        model.Add(cell_value[r][c] == base)


In [12]:
def add_region_NEQ(model, cells, cell_value):
    vars_ = [cell_value[r][c] for (r, c) in cells]
    model.AddAllDifferent(vars_)


In [13]:
def add_region_SUM(model, cells, target, cell_value):
    model.Add(sum(cell_value[r][c] for (r, c) in cells) == target)


In [14]:
def add_region_LT(model, cells, target, cell_value):
    model.Add(sum(cell_value[r][c] for (r, c) in cells) <= target)


In [15]:
def add_region_GT(model, cells, target, cell_value):
    model.Add(sum(cell_value[r][c] for (r, c) in cells) >= target)


In [16]:
def add_region_constraints(model, board, constraints, cell_value):
    n = len(board)

    # Build region → list of cells mapping
    regions = {}
    for r in range(n):
        for c in range(n):
            ch = board[r][c]
            if ch not in ('.', '#'):
                regions.setdefault(ch, []).append((r,c))

    # Add constraints
    for label, (ctype, value) in constraints.items():
        if label not in regions:
            continue
        cells = regions[label]

        if ctype == "EQ":
            add_region_EQ(model, cells, cell_value)

        elif ctype == "NEQ":
            add_region_NEQ(model, cells, cell_value)

        elif ctype == "SUM":
            add_region_SUM(model, cells, value, cell_value)

        elif ctype == "LT":
            add_region_LT(model, cells, value, cell_value)

        elif ctype == "GT":
            add_region_GT(model, cells, value, cell_value)

        else:
            raise ValueError(f"Unknown constraint type: {ctype}")

# Initializing Model and Solving the Puzzle

The solve_pips function will take in the board size, the board, the constraints, and the dominos (which our file parser sets up earlier).
Then, it generates teh variables and constraints and solves the puzzle

In [17]:
def solve_pips(n, board, constraints, dominos):
    model = cp_model.CpModel()

    # Variables & placements
    place, placements, cell_value, maxpip = define_variables(model, n, board, dominos)

    # Constraints:
    # 1. Every valid cell is covered exactly once
    is_place = add_cell_coverage_constraints(model, n, board, place, placements, cell_value)

    # 2. Domino placement ⇒ pip values in the two cells
    add_domino_value_constraints(model, n, placements, place, is_place, cell_value)

    # 3. Region constraints
    add_region_constraints(model, board, constraints, cell_value)

    # Solve
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status not in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        print("No solution found.")
        return None

    # Build solution grid of pip values (or None)
    sol_grid = [[None] * n for _ in range(n)]
    for r in range(n):
        for c in range(n):
            if board[r][c] == '.':
                sol_grid[r][c] = None
            else:
                sol_grid[r][c] = solver.Value(cell_value[r][c])

    return sol_grid, solver


Solving the puzzle we parsed in earlier:

In [18]:
sol, solver = solve_pips(n, board, constraints, dominos)
if sol is not None:
    print("Solution pip grid:")
    for row in sol:
        print(" ".join("." if v is None else str(v) for v in row))
    print(solver.ResponseStats())

Solution pip grid:
3 0 1 5
. . . 6
. . 5 5
. . 5 .
CpSolverResponse summary:
status: OPTIMAL
objective: 0
best_bound: 0
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 0.00919417
usertime: 0.00919568
deterministic_time: 9.76e-06
gap_integral: 0
solution_fingerprint: 0x27e5732c64fe3957



# Benchmarking
Now that we've shown our solver works, we can try doing some basic benchmarking using this solver and the `create_puzzle.py` files.

In [ ]:
import time
import os
import numpy as np
from create_puzzle import encode

NUM_PUZZLES = 100

# ---------------------------------------------
# Create output folder for storing puzzles+solutions
# ---------------------------------------------
RESULT_DIR = "_puzzle_results"
os.makedirs(RESULT_DIR, exist_ok=True)

gen_times = []
solve_times = []
successes = 0


def write_solution_file(path, board, sol):
    """Write solved pip board as text."""
    with open(path, "w") as f:
        if sol is None:
            f.write("NO SOLUTION\n")
            return
        n = len(sol)
        for r in range(n):
            line = " ".join("." if sol[r][c] is None else str(sol[r][c]) 
                            for c in range(n))
            f.write(line + "\n")

for i in range(NUM_PUZZLES):
    print(f"Puzzle {i+1}/{NUM_PUZZLES}")

    # -------------------------
    # 1. Generate puzzle
    # -------------------------
    t0 = time.time()
    puzzle_text = encode()    
    t1 = time.time()
    gen_times.append(t1 - t0)

    # Save puzzle text
    puzzle_path = os.path.join(RESULT_DIR, f"puzzle_{i}.txt")
    with open(puzzle_path, "w") as f:
        f.write(puzzle_text)

    # -------------------------
    # 2. Parse & solve puzzle
    # -------------------------

    sol = None
    solver = None

    # try:
    n, board, constraints, dominos = parse_pips_puzzle(puzzle_path)
    t2 = time.perf_counter()
    sol, solver = solve_pips(n, board, constraints, dominos)

    if sol is not None:
        successes += 1

    # except Exception as e:
    #     print(f"❌ Puzzle {i+1} failed with error: {e}")

    t3 = time.perf_counter()
    solve_times.append(t3 - t2)

    # -------------------------
    # 3. Save solution to file
    # -------------------------
    solution_path = os.path.join(RESULT_DIR, f"solution_{i}.txt")
    write_solution_file(solution_path, board, sol)


# --------------------------------------
# RESULTS
# --------------------------------------

print("\n========== BENCHMARK RESULTS ==========")
print(f"Total puzzles: {NUM_PUZZLES}")
print(f"Solved puzzles: {successes}/{NUM_PUZZLES}\n")

print(f"Average puzzle generation time: {np.mean(gen_times):.4f} seconds")
print(f"Median puzzle generation time:  {np.median(gen_times):.4f}")

print(f"Average solve time:             {np.mean(solve_times):.4f} seconds")
print(f"Median solve time:              {np.median(solve_times):.4f}")

print(f"Total benchmark time:           {sum(gen_times) + sum(solve_times):.2f} seconds")
print("=========================================")

Puzzle 1/100
Puzzle 2/100
Puzzle 3/100
Puzzle 4/100
Puzzle 5/100
Puzzle 6/100
Puzzle 7/100
Puzzle 8/100
Puzzle 9/100
Puzzle 10/100
Puzzle 11/100
Puzzle 12/100
Puzzle 13/100
Puzzle 14/100
Puzzle 15/100
Puzzle 16/100
Puzzle 17/100
Puzzle 18/100
Puzzle 19/100
Puzzle 20/100
Puzzle 21/100
Puzzle 22/100
Puzzle 23/100
Puzzle 24/100
Puzzle 25/100
Puzzle 26/100
Puzzle 27/100
Puzzle 28/100
Puzzle 29/100
Puzzle 30/100
Puzzle 31/100
Puzzle 32/100
Puzzle 33/100
Puzzle 34/100
Puzzle 35/100
Puzzle 36/100
Puzzle 37/100
Puzzle 38/100
Puzzle 39/100
Puzzle 40/100
Puzzle 41/100
Puzzle 42/100
Puzzle 43/100
Puzzle 44/100
Puzzle 45/100
Puzzle 46/100
Puzzle 47/100
Puzzle 48/100
Puzzle 49/100
Puzzle 50/100
Puzzle 51/100
Puzzle 52/100
Puzzle 53/100
Puzzle 54/100
Puzzle 55/100
Puzzle 56/100
Puzzle 57/100
Puzzle 58/100
Puzzle 59/100
Puzzle 60/100
Puzzle 61/100
Puzzle 62/100
Puzzle 63/100
Puzzle 64/100
Puzzle 65/100
Puzzle 66/100
Puzzle 67/100
Puzzle 68/100
Puzzle 69/100
Puzzle 70/100
Puzzle 71/100
Puzzle 72/100
P